## 碳交易敏感度

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

data = pd.read_excel("fitting_production_data_optimal.xlsx", sheet_name = 0)
data.head()

,Order,Quantity,DB Setup Time (min),DB Process Time (UPH),WB Setup Time (min),WB Process Time (UPH),Weight,Due Date,DB Setup Cost,DB Production cost,WB Setup Cost,WB Production Cost,Unnamed: 12,DB min,WB min,Remain min
0,0,15944,50,39,50,66,6,6573,30,148,52,258,NaN,408.820513,241.575758,5922.603730
1,1,40312,50,21,50,40,6,7457,18,92,65,327,NaN,1919.619048,1007.800000,4529.580952
2,2,44835,50,25,50,45,5,5738,60,299,76,381,NaN,1793.400000,996.333333,2948.266667
3,3,24581,50,1,50,3,8,10863,45,224,78,388,NaN,24581.000000,8193.666667,-21911.666667
4,4,50361,50,40,50,69,3,14659,45,226,75,373,NaN,1259.025000,729.869565,12670.105435


In [2]:
# Create a new model
model = gp.Model("carbon_tax")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19


In [3]:
# Set parameters
order1 = 30 # Numbers of order
order2 = 30
process = 2  # Numbers of process
machine_process1 = 15 # Numbers of machine for process 1
machine_process2 = 20 # Numbers of machine for process 2
M = 1000000  # A very large number
W = np.array(data.iloc[:order1, 6])  # Order weight
Q = np.array(data.iloc[:order1, 1])  # Quantity
D = np.array(data.iloc[:order1, 7])  # Due date
S = np.zeros((order1, process, max(machine_process1, machine_process2)))  # Setup time
for i1 in range(order1):
    S[i1, 0, :machine_process1] = data.iloc[i1, 2]
    S[i1, 1, :machine_process2] = data.iloc[i1, 4]

V = np.zeros((order1, process, max(machine_process1, machine_process2)))  # Process time
for i1 in range(order1):
    V[i1, 0, :machine_process1] = data.iloc[i1, 3]
    V[i1, 1, :machine_process2] = data.iloc[i1, 5]

PC = np.zeros((order1, process, max(machine_process1, machine_process2)))  # Production cost
for i1 in range(order1):
    PC[i1, 0, :machine_process1] = data.iloc[i1, 8]
    PC[i1, 1, :machine_process2] = data.iloc[i1, 9] 

SC = np.zeros((order1, process, max(machine_process1, machine_process2)))  # Setup cost
for i1 in range(order1):
    SC[i1, 0, :machine_process1] = data.iloc[i1, 10]
    SC[i1, 1, :machine_process2] = data.iloc[i1, 11]


Phi_hat = np.zeros((process, max(machine_process1, machine_process2)))  # Production rate which invest the machine
for i1 in range(order1):
    Phi_hat[0, :machine_process1] = 1.1
    Phi_hat[1, :machine_process2] = 1.1

KW = np.array([1.3, 2.4])  # Machine power rate
Beta = np.array([24012, 24012])  # Investment cost on increase production rate
CEF = 0.000495  # Carbon emission factor
GCE = 130 # Government carbon emission cap
CP = 300  # Carbon price
Alpha = 84  # When the completion time isn't satisfied, it will be fined
Delta = np.array([37258, 37258])# Investment cost on increase energy-efficent
Eplison = 0.7

In [4]:
# Create decision variables
x = model.addVars(order1, process, max(machine_process1, machine_process2), vtype=GRB.BINARY, name='x')
y = model.addVars(order1, order2, process, max(machine_process1, machine_process2), vtype=GRB.BINARY, name='y')
z = model.addVars(order1, order2, process, max(machine_process1, machine_process2), vtype=GRB.BINARY, name='z')
f = model.addVars(order1, process, lb=0, vtype=GRB.CONTINUOUS, name='f')
c = model.addVars(order1, lb=0, vtype=GRB.CONTINUOUS, name='c')
st = model.addVars(order1, process, max(machine_process1, machine_process2), lb=0, vtype=GRB.CONTINUOUS, name='st')
pt = model.addVars(order1, process, max(machine_process1, machine_process2), lb=0, vtype=GRB.CONTINUOUS, name='pt')
sm = model.addVars(order1, process, lb=0, vtype=GRB.CONTINUOUS, name='sm')
pm = model.addVars(order1, process, lb=0, vtype=GRB.CONTINUOUS, name='pm')
ec = model.addVars(order1, process, max(machine_process1, machine_process2), lb=0, vtype=GRB.CONTINUOUS, name='ec')
e = model.addVars(order1, process, max(machine_process1, machine_process2), lb = 0, vtype = GRB.CONTINUOUS, name='e')
em = model.addVars(order1, process, max(machine_process1, machine_process2), lb = 0, vtype = GRB.CONTINUOUS, name='em')
cce = model.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'cce')
w = model.addVar(vtype = GRB.BINARY, name = 'w')
phi = model.addVars(process, max(machine_process1, machine_process2), vtype = GRB.BINARY, name = 'phi')
eta = model.addVars(process, max(machine_process1, machine_process2), vtype=GRB.BINARY, name='eta')  # Whether to invest on machine
ept = model.addVars(order1, process, max(machine_process1, machine_process2), lb=0, vtype=GRB.CONTINUOUS, name='ept')
TC = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC')
TC1 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC1')
TC2 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC2')
TC3 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC3')
TC4 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC4')
TC5 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC5')
TC6 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC6')
TC7 = model.addVar(vtype = GRB.CONTINUOUS, name = 'TC7')

In [5]:
######## The function of Production time ########
P = {}
for i1 in range(order1):
    for j  in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            P[i1, j, k] = (Q[i1] / (V[i1, j, k] * Phi_hat[j, k])) * phi[j, k] + (Q[i1] / V[i1, j, k]) * (1 - phi[j, k])

            
######## The function of Buying or Selling Carbon Credit ########
#BCC = cce - GCE  # Buy credits
#SCC = GCE - cce  # Sell credits


######## The function of electric used ########
E = {}
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            E[i1, j, k] = KW[j] * Eplison * eta[j, k] + KW[j] * (1 - eta[j, k])

In [6]:
# Set objective function
model.setObjective(TC1 + TC2 + TC3 + TC4  + TC6 +TC7  , GRB.MINIMIZE)

In [7]:
######## Production Schedule constraints ########
# constraint 1
for i1 in range(order1):
    model.addConstr(c[i1] >= f[i1, process - 1], 'c1_%d' % i1)

# constraint 2
for i1 in range(order1):
    for j in range(process):
        model.addConstr(f[i1, j] >= pm[i1, j] + gp.quicksum(P[i1, j, k] * x[i1, j, k] for k in range(machine_process1 if j == 0 else machine_process2)),
                        'c2_%d_%d' % (i1, j))
        sm[i1, j + 1] = f[i1, j]

# constraint 3
for i1 in range(order1):
    for j in range(process):
        model.addConstr(pm[i1, j] >= sm[i1, j] + gp.quicksum(S[i1, j, k] * x[i1, j, k] for k in range(machine_process1 if j == 0 else machine_process2)), 'c3_%d_%d' % (i1, j))

# constraint 4
for i1 in range(order1):
    for j in range(process):
        model.addConstr(gp.quicksum(x[i1, j, k] for k in range(machine_process1 if j == 0 else machine_process2)) == 1, 'c4_%d_%d' % (i1, j))

# constraint 5
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            model.addConstr(sm[i1, j] - M * (1 - x[i1, j, k]) <= st[i1, j, k], 'c5_%d_%d_%d' % (i1, j, k))

# constraint 6:
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            model.addConstr(pt[i1, j, k] == pm[i1, j] * x[i1, j, k], 'c6_%d_%d_%d' % (i1, j, k))

# constraint 7.1
for i1 in range(order1):
    for i2 in range(order2):
        if i1 != i2:
            for j in range(process):
                for k in range(machine_process1 if j == 0 else machine_process2):
                    model.addConstr(y[i1, i2, j, k] <= 0.5 * (x[i1, j, k] + x[i2, j, k]), 'c7.1_%d_%d_%d_%d' % (i1, i2, j, k))

# constraint 7.2
for i1 in range(order1):
    for i2 in range(order2):
        if i1 != i2:
            for j in range(process):
                for k in range(machine_process1 if j == 0 else machine_process2):
                    model.addConstr(0.5 * (x[i1, j, k] + x[i2, j, k]) <= y[i1, i2, j, k] + 0.5, 'c7.2_%d_%d_%d_%d' % (i1, i2, j, k))

# constraint 7.3
for i1 in range(order1):
    for i2 in range(order2):
        if i1 != i2:
            for j in range(process):
                for k in range(machine_process1 if j == 0 else machine_process2):
                    model.addConstr(z[i1, i2, j, k] + z[i2, i1, j, k] == y[i1, i2, j, k], 'c7.3_%d_%d_%d_%d' % (i1, i2, j, k))

# constraint 8
for i1 in range(order1):
    for j in range(process):
        lhs = gp.quicksum(z[i2, i1, j, k] for i2 in range(order2) if i1 != i2 for k in range(max(machine_process1, machine_process2)))
        rhs = gp.quicksum(x[i1, j, k] for k in range(machine_process1 if j == 0 else machine_process2))
        model.addConstr(lhs <= rhs, 'c8_%d_%d' % (i1, j))

# constraint 9
for i1 in range(order1):
    for j in range(process):
        lhs = gp.quicksum(z[i1, i2, j, k] for i2 in range(order2) if i1 != i2 for k in range(max(machine_process1, machine_process2)))
        rhs = gp.quicksum(x[i1, j, k] for k in range(machine_process1 if j == 0 else machine_process2))
        model.addConstr(lhs <= rhs, 'c9_%d_%d' % (i1, j))

# constraint 10
for i1 in range(order1):
    for i2 in range(order2):
        if i1 != i2:
            for j in range(process):
                for k in range(machine_process1 if j == 0 else machine_process2):
                    model.addConstr(pt[i1, j, k] + (P[i1, j, k] * z[i1, i2, j, k]) - M * (1 - z[i1, i2, j, k]) <= sm[i2, j], 'c10_%d_%d_%d_%d' % (i1, i2, j, k))

#------------------------------------------------------------------------------------------------------------------------------------------------------------

# constraint 11:
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            model.addConstr(ept[i1, j, k] >= x[i1, j, k] * (P[i1, j, k] + S[i1, j, k]) / 60, 'c11_%d_%d_%d' % (i1, j, k))

# constraint 12:
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
                        model.addConstr(ec[i1, j, k] >= ept[i1, j, k] * E[i1, j, k], 'c12_%d_%d_%d' % (i1, j, k))

# constraint 13: 
for i1 in range(order1):
    for j in range(process):
        for k in range(machine_process1 if j == 0 else machine_process2):
            model.addConstr(em[i1, j, k] >= CEF * ec[i1, j, k], 'c13_%d_%d_%d' % (i1, j, k))

            
# constraint 14:
model.addConstr(cce >= gp.quicksum(em[i1, j, k] for i1 in range(order1) for j in range(process) for k in range(machine_process1 if j == 0 else machine_process2)), 'c14')

# constraint 15 & 16:
#model.addConstr(cce <= GCE + M * w, 'c15')
#model.addConstr(cce >= GCE + M * (w - 1), 'c16')


######## Invest Productivity Machines constraints ########
# constraint 17:
for j in range(process):
     for k in range(max(machine_process1, machine_process2)-1):
        model.addConstr(phi[j, k+1] >= phi[j, k], 'c17_%d_%d' % (j, k))

# constraint 18:
for j in range(process):
    for k in range(machine_process1-1 if j == 0 else machine_process2-1):
        model.addConstr(phi[j, k] >= phi[j, k+1], 'c18_%d_%d' % (j, k))


######## Order Fulfill constraints ########
# constraint 19:
for i1 in range(order1):
    model.addConstr(c[i1] - D[i1] >= 0, 'c19_%d' % i1)
    

######## Invest Energy-saving Machines constraints ########
# constraint 20:
for j in range(process):
    for k in range(machine_process1-1 if j == 0 else machine_process2-1):
        model.addConstr(eta[j, k+1] >= eta[j, k], 'c20_%d_%d' % (j, k))

# constraint 21:
for j in range(process):
    for k in range(machine_process1-1 if j == 0 else machine_process2-1):
        model.addConstr(eta[j, k] >= eta[j, k+1], 'c21_%d_%d' % (j, k))


######## Cost constraints ########
model.addConstr(TC == TC1 + TC2 + TC3 + TC4 +TC6,'c22')  # Total cost
model.addConstr(TC1 == gp.quicksum(PC[i1, j, k] * Q[i1] * x[i1, j, k] for i1 in range(order1) for j in range(process) for k in range(machine_process1 if j == 0 else machine_process2)),'c23')  # Production cost
model.addConstr(TC2 == gp.quicksum(SC[i1, j, k] * z[i1, i2, j, k] for i1 in range(order1) for i2 in range(order2) if i1 != i2 for j in range(process)for k in range(machine_process1 if j == 0 else machine_process2)), 'c24')  # Setup cost
model.addConstr(TC3 == Alpha * gp.quicksum(W[i1] * (c[i1] - D[i1]) for i1 in range(order1)),'c25')  # Order fulfill penalty
model.addConstr(TC4 == CP * cce , 'c26')  # Carbon Cap and Trade
#model.addConstr(TC5 == CP * SCC * (1 - w), 'c27')  # Carbon Cap and Trade
model.addConstr(TC6 == gp.quicksum(Beta[j] * phi[j, k] for j in range(process) for k in range(machine_process1 if j == 0 else machine_process2)),'c28')  # Investment cost on production rate
model.addConstr(TC7 == gp.quicksum(Delta[j] * eta[j, k] for j in range(process) for k in range(machine_process1 if j == 0 else machine_process2)),'c29')  # Investment cost on energy-efficent


"""---------- New ----------"""
######## Order fulfill rate constraints ######## 

r = model.addVars(order1, vtype=GRB.BINARY, name='r')
G = order1  # Number of order

# constraint 30:
for i1 in range(order1):
    model.addConstr(c[i1] - D[i1] <= M * r[i1], 'c30_%d' % i1) 

# constraint 31:
for i1 in range(order1):
    model.addConstr(c[i1] - D[i1] >= M * (r[i1] - 1), 'c31_%d' % i1)


# constraint 32:
model.addConstr(gp.quicksum(r[i1] for i1 in range(order1)) <= G * 0.2, 'c32')

"""---------- End of update ----------"""

#Update the model
model.update()

In [8]:
# Set the time limit
model.setParam('TimeLimit', 4 * 60 * 60)

# Set the gap to 0.01
model.setParam('MIPGap', 1e-2)

# Set MIPFocus to 1
model.setParam('MIPFocus', 1)

Set parameter TimeLimit to value 14400
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


In [9]:
# Optimize the model
model.optimize()

# Create lp file
#model.write("carbon_trade.lp")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 93956 rows, 80730 columns and 386398 nonzeros
Model fingerprint: 0x0a71b240
Model has 33660 quadratic constraints
Variable types: 7419 continuous, 73311 integer (73311 binary)
Coefficient statistics:
  Matrix range     [5e-04, 2e+07]
  QMatrix range    [4e-01, 2e+03]
  QLMatrix range   [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+08]
  QRHS range       [1e+06, 1e+06]
Presolve removed 62141 rows and 45874 columns
Presolve time: 2.32s
Presolved: 132254 rows, 70351 columns, 460931 nonzeros
Presolved model has 2100 SOS constraint(s)
Variable types: 6472 continuous, 63879 integer (63879 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Sho

In [10]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Numbers of variables: ", model.numVars)
    print("Numbers of constraints: ", model.numConstrs)
    print("Objective value: ", round(model.objVal, 0))
    print("Node Count: ", round(model.NodeCount, 0))
    print("Simplex Iterations: ", round(model.getAttr(GRB.Attr.IterCount), 0))
    print("Runtime: ", round(model.Runtime, 2), 'sec')
#    for v in model.getVars():
#        if v.vtype == GRB.INTEGER:
#            print("%s = %d" % (v.varName, v.x))
#        else:
#            print("%s = %.2f" % (v.varName, v.x))
else:
    print("No solution found.")

Optimal solution found!
Numbers of variables:  80730
Numbers of constraints:  93956
Objective value:  298835107.0
Node Count:  1.0
Simplex Iterations:  13648.0
Runtime:  81.43 sec


In [11]:
######## Order fulfill rate ######## 
gamma = 0
for i1 in range(order1):
    if (c[i1].x <= D[i1]):
        gamma += 1

print("Order fulfill rate = %.2f%%" % ((gamma / order1) * 100))  


######## Cost ########
print('\nCost')
print("Total cost = %.0f" % TC.x)  
print("Production cost = %.0f" % TC1.x)  
print("Setup cost = %.0f" % TC2.x)
print("Order fulfill penalty = %.0f" % TC3.x)
print("Carbon Cap and Trade (buy) = %.0f" % TC4.x)
print("Carbon Cap and Trade (sell) = %.0f" % TC5.x)
print("Investment cost on increase production rate = %.0f" % TC6.x)
print("Investment cost on increase energy-efficent = %.0f" % TC7.x)


######## Cost Ratio ########
print('\nCost ratio')
print("Production cost = %.2f%%" % ((TC1.x / TC.x) * 100))  
print("Setup cost = %.2f%%" % ((TC2.x / TC.x) * 100))
print("Order fulfill penalty = %.2f%%" % ((TC3.x / TC.x) * 100))
print("Carbon Cap and Trade (buy) = %.2f%%" % (((TC4.x) / TC.x) * 100))
print("Carbon Cap and Trade (sell) = %.2f%%" % (((TC5.x) / TC.x) * 100))
print("Investment cost on increase production rate = %.2f%%" % ((TC6.x / TC.x) * 100))  
print("Investment cost on increase energy-efficent = %.2f%%" % ((TC7.x / TC.x) * 100))  


######## c_max ########
c_time = []
for i1 in range(order1):
    c_time.append(c[i1].x)

for i1 in range(order1 - 1):
    if (c_time[i1] - c_time[i1 + 1] >= 0):
        c_max = c_time[i1]
        c_time[i1 + 1] = c_max
    else:
        c_max = c_time[i1 + 1]

print("Latest finish time = %.2f" % c_max, 'sec')

Order fulfill rate = 80.00%

Cost
Total cost = 298835107
Production cost = 266981941
Setup cost = 2600
Order fulfill penalty = 31009408
Carbon Cap and Trade (buy) = 738
Carbon Cap and Trade (sell) = 0
Investment cost on increase production rate = 840420
Investment cost on increase energy-efficent = 0

Cost ratio
Production cost = 89.34%
Setup cost = 0.00%
Order fulfill penalty = 10.38%
Carbon Cap and Trade (buy) = 0.00%
Carbon Cap and Trade (sell) = 0.00%
Investment cost on increase production rate = 0.28%
Investment cost on increase energy-efficent = 0.00%
Latest finish time = 32195.91 sec


In [12]:
cce

<gurobi.Var cce (value 2.4592116856440374)>

In [13]:
result = {
    'TC': [TC.x],
    'TC1': [TC1.x],
    'TC2': [TC2.x],
    'TC3': [TC3.x],
    'TC4': [TC4.x],
    'TC5': [TC5.x],
    'TC6': [TC6.x],
    'TC7': [TC7.x],
    'Phi_hat': [Phi_hat],
    'Eplison': [Eplison],
    'KW': [KW],
    'Beta': [Beta],
    'Delta': [Delta],
    'CEF': [CEF],
    'GCE': [GCE],
    'cce':[cce.x],
    'CP': [CP],
    'Alpha': [Alpha],
    'c_max':[c_max],
    'Order fulfill rate':[(gamma / order1) * 100],
    "Production cost ": [(TC1.x / TC.x) * 100],
    "Setup cost ": [(TC2.x / TC.x) * 100],
    "Order fulfill penalty ": [(TC3.x / TC.x) * 100],
    "Carbon Cap and Trade (buy) ": [((TC4.x) / TC.x) * 100],
    "Carbon Cap and Trade (sell) ": [((TC5.x) / TC.x) * 100],    
    "Investment cost on increase production rate ": [(TC6.x / TC.x) * 100],
    "Investment cost on increase energy-efficent ": [(TC7.x / TC.x) * 100],
    'Node Count':[model.NodeCount],
    'Numbers of variables':[model.numVars],
    'Numbers of constraints':[model.numConstrs],
    'Simplex Iterations':[model.getAttr(GRB.Attr.IterCount)],
    'Runtime':[model.Runtime]
}

df = pd.DataFrame(result)
df_transposed = df.transpose()

df_transposed.to_excel("M2.xlsx", header=False, engine='xlsxwriter')